# Wikipics demo

In [1]:
from pathlib import Path
from typing import NamedTuple
from ipywidgets.widgets import Image, Layout, HBox

import httpx

from wikipics import get_sample_url, get_sample_urls

class ImageRecord(NamedTuple):
    pixels: bytes
    name: str
    size: int

In [2]:
def fetch(url) -> ImageRecord:
    resp = httpx.get(url)
    resp.raise_for_status()
    name = Path(url).name
    img = ImageRecord(resp.content, name, len(resp.content))
    print(f'{img.size:12_} bytes | {img.name}')
    return img

In [3]:
#img_rec = fetch(get_sample_url(1_000_000))
#Image(value=img_rec.pixels)

In [4]:
%%time
img_widgets = []
qty = 10
scale = f'{100//qty}%'
urls = get_sample_urls(1_000_000, qty)
total_bytes = 0
for url in urls:
    img_rec = fetch(url)
    total_bytes += img_rec.size
    img_widgets.append(Image(value=img_rec.pixels, layout=Layout(width=scale, height=scale)))

print(f'TOTAL BYTES: {total_bytes:_}')
HBox(img_widgets)

   1_002_230 bytes | Squash_bug_Coreidae_hz.jpg
   1_006_040 bytes | Elmer_Chickering_-_John_Philip_Sousa.jpg
     968_342 bytes | Anolemeal6127.jpg
   1_007_852 bytes | Chrysopilus_Snipe_fly.jpg
     970_664 bytes | Ants_eating_cicada%2C_jjron_22.11.2009.jpg
   4_458_998 bytes | Mycteria_leucocephala_-_Pak_Thale.jpg
     983_318 bytes | Hippo_skull_dark.jpg
   1_007_715 bytes | White-headed_dwarf_gecko.jpg
     986_062 bytes | Perga_sp._AF_2_edit1.jpg
   1_012_241 bytes | Vanellus_miles_novaehollandiae.jpg
TOTAL BYTES: 13_403_462
CPU times: user 692 ms, sys: 218 ms, total: 909 ms
Wall time: 15.1 s


In [5]:
class Gallery:
    def __init__(self, num_images):
        scale = f'{100//num_images}%'
        self.img_layout = Layout(width=scale, height=scale)
        no_img = open('no-image.png', 'rb').read()
        self.img_widgets = [Image(value=no_img, layout=self.img_layout)] * num_images
        self.box = HBox(self.img_widgets)
        self.size = 0

    def display(self):
        display(self.box)

    def update(self, index, pixels):
        self.size += len(pixels)
        self.img_widgets[index] = Image(value=pixels, layout=self.img_layout)
        self.box.children = self.img_widgets

In [6]:
%%time
num_images = 10
gallery = Gallery(num_images)
gallery.display()

urls = get_sample_urls(1_000_000, num_images)

for i, url in enumerate(urls):
    img_rec = fetch(url)
    total_bytes += img_rec.size
    gallery.update(i, img_rec.pixels)

print(f'TOTAL BYTES: {gallery.size:_}')

   1_000_424 bytes | Mesa-Verde---Cliff-Palace-in_1891_-_edit1.jpg
     992_514 bytes | June_odd-eyed-cat.jpg
   1_006_040 bytes | Elmer_Chickering_-_John_Philip_Sousa.jpg
     974_570 bytes | Los_Angeles_Pollution.jpg
     984_345 bytes | Queenmaryformalportrait_edit3.jpg
     989_420 bytes | STS-132_combined_launch_photos.jpg
   1_008_570 bytes | Blue-Mountains-Tree-Frog444.jpg
     983_492 bytes | Sphegina_montana_Syrphidae.jpg
     979_323 bytes | AgamaSinaita01_ST_10_edit.jpg
   1_012_241 bytes | Vanellus_miles_novaehollandiae.jpg
TOTAL BYTES: 9_930_939
CPU times: user 662 ms, sys: 190 ms, total: 852 ms
Wall time: 14.7 s


In [7]:
%%time
from concurrent import futures

num_images = 10
gallery = Gallery(num_images)
gallery.display()

urls = get_sample_urls(5_000_000, num_images)
with futures.ThreadPoolExecutor() as pool:
    img_records = pool.map(fetch, urls)
    for i, img_rec in enumerate(img_records):
        gallery.update(i, img_rec.pixels)

print(f'TOTAL BYTES: {gallery.size:_}')

   5_069_241 bytes | Sultan_Mehmed_IV_%282%29.jpg
   4_916_334 bytes | Sir_Joseph_Noel_Paton_-_The_Quarrel_of_Oberon_and_Titania_-_Google_Art_Project_2.jpg
   4_943_441 bytes | Lee_Bollinger_-_Daniella_Zalcman_less_noise.jpg
   5_074_252 bytes | Chromium_crystals_and_1cm3_cube.jpg
   4_926_858 bytes | Idi_Amin_caricature2.jpg
   5_034_423 bytes | Black-headed_Heron_%28Ardea_melanocephala%29.jpg
   4_955_510 bytes | Haeckel_Stephoidea_edit.jpg
   5_037_810 bytes | GSskater.jpg
   5_084_186 bytes | Klara_kyrka_february_2013_01.jpg
   4_987_730 bytes | Cigarette_smuggling_with_a_book.JPG
TOTAL BYTES: 50_029_785
CPU times: user 9.49 s, sys: 390 ms, total: 9.88 s
Wall time: 3.27 s


In [8]:
%%time
from IPython.display import display
from concurrent import futures

num_images = 10
gallery = Gallery(num_images)
gallery.display()
urls = get_sample_urls(1_000_000, num_images)

with futures.ThreadPoolExecutor() as pool:
    tasks = [pool.submit(fetch, url) for url in urls]
    for i, future in enumerate(futures.as_completed(tasks)):
        img_rec = future.result()
        gallery.update(i, img_rec.pixels)
        
print(f'TOTAL BYTES: {gallery.size:_}')

   1_008_846 bytes | Zebra_portrait.jpg
     983_492 bytes | Sphegina_montana_Syrphidae.jpg
     993_296 bytes | Soybean_cyst_nematode_and_egg_SEM.jpg
   1_007_715 bytes | White-headed_dwarf_gecko.jpg
   1_005_122 bytes | PikiWiki_Israel_13773_AIDA_AT_MASADA_2011.jpg
   1_000_617 bytes | Illustration_Punica_granatum2.jpg
   1_002_230 bytes | Squash_bug_Coreidae_hz.jpg
     968_342 bytes | Anolemeal6127.jpg
     977_947 bytes | Carambola_Starfruit.jpg
   1_007_836 bytes | Ardea_picata.jpg
TOTAL BYTES: 9_955_443
CPU times: user 7.38 s, sys: 167 ms, total: 7.55 s
Wall time: 2.26 s
